In [ ]:
import pkuseg
import pandas as pd
col=7
seg = pkuseg.pkuseg()           # 以默认配置加载模型

In [ ]:
def stopwords_(filepath, docs):
    lenth=len(docs)
    with open(filepath, 'r', encoding='utf-8') as f:
        stopwords = f.read().split('\n')
        # 删去docs中的停用词
        for i in range(0, lenth):
            #print(docs[i])
            for j in range(0, col):
                try:
                    k=0
                    while (k < len(docs[i][j])):  # for循环遍历分词后的每个词语
                        # 若字符在停用表中
                        if docs[i][j][k] in stopwords:
                            docs[i][j].remove(docs[i][j][k])
                        else:
                            k+=1
                except:
                    continue
    return docs

In [ ]:
#把word2doc保存到文件中
import pickle

def save_dict(dictionary, file_path):
    with open(file_path, 'wb') as file:
        pickle.dump(dictionary, file)


### 近义词合并

In [ ]:
# 构建近义词表
def synwords_(syn_path):
    synwords = []
    with open(syn_path, 'r', encoding='utf-8') as f:
        for line in f.readlines():
            split = line.strip().split(' ')
            number = split[0]
            templist = split[1:]
            if number[-1] == '=':
                synwords.append(templist)
    save_dict(synwords, './data_dict/synwords.pickle')
    return synwords

In [ ]:
def findsyn(word, synwords):
    if len(word) == 1:
        return -1
    for line in synwords:
        if word in line:
            return line
    return -1

In [ ]:
synwords = synwords_('./chinese_dictionary-master/dict_synonym.txt')
findsyn('犯罪', synwords)

In [ ]:
def syn_merge(docs,synwords):
    #将近义词替换为同义词列表第一个词
    #遍历分词、去停用词后的结果
    lenth=len(docs)
    for i in range(0, lenth):
        for j in range(0, col):
            for k in range(0, len(docs[i][j])):
                if findsyn(docs[i][j][k], synwords) != -1:
                    docs[i][j][k] = findsyn(docs[i][j][k], synwords)[0]
    return docs

In [ ]:
from pypinyin import lazy_pinyin
# 建立倒排索引表

def build(docs,syn=0):
    #syn=0不处理近义词
    #syn=1处理近义词
    if syn==1:
        docs=syn_merge(docs,synwords)
        #print("syn_merge_doc=")
        #print(docs)
    word2doc = {}
    for i in range(0, len(docs)):
        temp_freq = {}  # 记录当前文档中词频
        for j in range(0, col):
            for word in docs[i][j]:
                if word in temp_freq:
                    temp_freq[word] += 1
                else:
                    temp_freq[word] = 1
        # print(temp_freq)
        for word in temp_freq:
            if word in word2doc:
                word2doc[word].append((i, temp_freq[word]))
            else:
                word2doc[word] = [(i, temp_freq[word])]
    print(type(word2doc))
    word2doc = sorted(word2doc.items(), key=lambda x: lazy_pinyin(x[0]))
    # list转成dict
    word2doc = dict(word2doc)
    print(type(word2doc))
    return word2doc
# print(word2doc)
# word2doc按词的字母序排序
# word2doc = sorted(word2doc.items(),key=lambda x:x[0])
# print(word2doc)
# word2doc按汉语拼音首字母排序
# word2doc = sorted(word2doc.items(),key=lambda x:lazy_pinyin(x[0]))
# print(word2doc)

### 电影
#### 读取电影信息并分词

In [ ]:
# 读取movie/MovieDouban_0.csv文件第一列
df = pd.read_csv('./movie/MovieDouban_p0.csv',
                 encoding='utf-8', usecols=[4, 12, 13, 20, 21,7,9], header=0)
for num in range(1, 12):
    df1 = pd.read_csv('./movie/MovieDouban_p{}.csv'.format(num),
                      encoding='utf-8', usecols=[4, 12, 13, 20, 21,7,9], header=0)
    df = pd.concat([df, df1], axis=0, ignore_index=True)
# movie = df.to_string()
# print(movie)
# movie = seg.cut(movie)
# lenth=pd.value_counts(df).count()

movie_raw = df.values.tolist()
save_dict(movie_raw, './data_dict/movie_raw.pickle')
# movie_raw不随movie变化
movie = df.values.tolist()
lenth = len(movie)
print(lenth)
# print(movie[0][0])
# print(type(movie[0][0]))#str
for i in range(0, lenth):
    for j in range(0, col):
        # 大写转小写
        try:
            #去除·
            movie[i][j]=movie[i][j].replace('·','')
            movie[i][j] = movie[i][j].lower()
            movie[i][j] = seg.cut(movie[i][j])
        except:
            movie[i][j] = ''
        print(movie[i][j])
print(type(movie))
#print(movie)
#print(movie_raw)

#### 建立电影倒排表

In [ ]:
movie_s1 = stopwords_('./stopwords-master/hit_stopwords.txt', movie)
movie_s2 = stopwords_(
    './stopwords-master/baidu_stopwords.txt', movie_s1)
#print(movie_s2)
#print(movie_s2[97])
word2doc_movie = build(movie_s2,syn=1)
#print(word2doc)


### 书籍
#### 读取书籍信息并分词

In [ ]:
# 读取book/bookDouban_0.csv文件第一列
df = pd.read_csv('./book/BookDouban_p0.csv',
                 encoding='utf-8', usecols=[1, 5, 6, 14, 16,8,9], header=0)
for num in range(1, 12):
    df1 = pd.read_csv('./Book/bookDouban_p{}.csv'.format(num),
                      encoding='utf-8', usecols=[1, 5, 6, 14, 16,8,9], header=0)
    df = pd.concat([df, df1], axis=0, ignore_index=True)
# book = df.to_string()
# print(book)
# book = seg.cut(book)
# lenth=pd.value_counts(df).count()

book_raw = df.values.tolist()
save_dict(book_raw, './data_dict/book_raw.pickle')
# book_raw不随book变化
book = df.values.tolist()
lenth = len(book)
print(lenth)
# print(book[0][0])
# print(type(book[0][0]))#str
for i in range(0, lenth):
    for j in range(0, col):
        # 大写转小写
        try:
            #去除·
            book[i][j]=book[i][j].replace('·','')
            book[i][j] = book[i][j].lower()
            book[i][j] = seg.cut(book[i][j])
        except:
            book[i][j] = ''
print(type(book))
# print(book)
#print(book_raw)

#### 建立书籍倒排表

In [ ]:
book_s1 = stopwords_('./stopwords-master/hit_stopwords.txt', book)
book_s2 = stopwords_(
    './stopwords-master/baidu_stopwords.txt', book_s1)
#print(book_s2)
#print(book_s2[97])
word2doc_book = build(book_s2,syn=1)
#print(word2doc)

### 倒排表保存到文件

In [ ]:
# 保存字典到文件
save_dict(word2doc_movie, './data_dict/word2doc_movie.pickle')
save_dict(word2doc_book, './data_dict/word2doc_book.pickle')

In [ ]:
class TrieNode:
    def __init__(self):
        self.children = {}
        self.is_end_of_word = False

def insert(root, word):
    curr = root
    for char in word:
        if char not in curr.children:
            curr.children[char] = TrieNode()
        curr = curr.children[char]
    curr.is_end_of_word = True

def compress(strings):
    root = TrieNode()
    for string in strings:
        insert(root, string)
    
    compressed = []
    for string in strings:
        curr = root
        prefix = ''
        for char in string:
            if char not in curr.children or len(curr.children) > 1:
                break
            curr = curr.children[char]
            prefix += char
        suffix = string[len(prefix):]
        compressed.append((prefix, suffix))
    
    return compressed

def decompress(compressed):
    root = TrieNode()
    for prefix, suffix in compressed:
        insert(root, prefix + suffix)
    
    decompressed = []
    for prefix, suffix in compressed:
        curr = root
        string = prefix
        for char in suffix:
            if char not in curr.children:
                break
            curr = curr.children[char]
            string += char
        decompressed.append(string)
    
    return decompressed

# 示例用法
strings = ['apple', 'app', 'application', 'banana']
compressed = compress(strings)
print('Compressed:', compressed)

decompressed = decompress(compressed)
print('Decompressed:', decompressed)

### 按块压缩

In [ ]:
#二分查找
def binary_search(list, target):
    low = 0
    high = len(list) - 1

    while low <= high:
        mid = int((low + high) / 2)
        if list[mid][0] == target:
            return list[mid][1]
        elif list[mid][0] > target:
            high = mid - 1
        else:
            low = mid + 1
    return -1


In [ ]:
import pickle
def zipped_dict(dictpath):
    key_zipped=""
    list_zipped={}
    ptr=0
    with open(dictpath, 'rb') as f:
        dict_movie=pickle.load(f)
        for key in dict_movie:
            #if(key=='活'):
            #    print(key,ptr)
            #print(ptr)
            key_zipped+=key
            list_zipped[ptr]=dict_movie[key]
            ptr=ptr+len(key)
    print(key_zipped)
    print(list_zipped)
    #保存到文件
    
    save_dict(key_zipped,'./data_dict/key_zipped_{}.pickle'.format(dictpath[21:-7]))
    save_dict(list_zipped,'./data_dict/list_zipped_{}.pickle'.format(dictpath[21:-7]))

In [ ]:
#在压缩字典中查找
def find_zipped(key_zipped,list_zipped,word):
    index=0
    if word in key_zipped:
        while(index<len(key_zipped)):
            index=key_zipped[index+1:].find(word)+index+1#第一次出现的位置
            print(word,index)
            if index in list_zipped:
                #print('--------------------')
                return list_zipped[index]
    else:
        return []
    

In [ ]:
zipped_dict('./data_dict/word2doc_movie.pickle')
zipped_dict('./data_dict/word2doc_book.pickle')

In [ ]:
def load_plk(filepath):
    with open(filepath, 'rb') as file:
        return pickle.load(file)

In [ ]:
key_zipped=load_plk('./data_dict/key_zipped_movie.pickle')
list_zipped=load_plk('./data_dict/list_zipped_movie.pickle')
print(list_zipped)
dict_movie=load_plk('./data_dict/word2doc_movie.pickle')
print(find_zipped(key_zipped,list_zipped,'活'))
#print(type(key_zipped[9574:]))
#print(key_zipped[9574:].find("活"))
#print(key_zipped[9573:9575])
print(dict_movie["活"])#第8个，索引24

In [ ]:
from pygtrie import Trie

# 构建前缀树
trie = Trie()
trie['我懂'] = 1
trie['我'] = 2
trie['我爱你'] = 2
trie['orange'] = 3

# 查询前缀树
print(trie.get('我'))  # 输出: 1
print(trie.keys(prefix='我'))  # 输出: ['banana']

In [ ]:
import pickle
from pygtrie import Trie

# 构建前缀树
def trie_zip(dictpath):
    with open(dictpath, 'rb') as f:
        dict_movie=pickle.load(f)
    trie = Trie()
    for word in dict_movie:
        trie[word] = dict_movie[word]
    save_dict(trie,'./data_dict/gtrie_{}.pickle'.format(dictpath[21:-7]))

trie_zip('./data_dict/word2doc_movie.pickle')

In [ ]:
from pytrie import StringTrie

# 构建前缀树
def trie_zip(dictpath):
    with open(dictpath, 'rb') as f:
        dict_movie=pickle.load(f)
    trie = StringTrie()
    for word in dict_movie:
        trie[word] = dict_movie[word]
    save_dict(trie,'./data_dict/trie_{}.pickle'.format(dictpath[21:-7]))

trie_zip('./data_dict/word2doc_movie.pickle')